# Generate statistics

## Run this for both HiRES or CHARM experiments

In [ ]:
library(tidyverse)
library(ggpubr)
library(yaml)
library(patchwork)

config <- read_yaml(file = "../CHARM_preprocess_pipeline/config.yaml")

to_gigabases <- function(raw_bp) {
  raw_bp / 4 * 300 / 1e9
}

extract_sample <- function(path, slice_position, strip_after = NULL) {
  sample <- str_split(path, "/", simplify = TRUE)[, slice_position]
  if (!is.null(strip_after)) {
    sample <- str_split(sample, strip_after, simplify = TRUE)[, 1]
  }
  sample
}

read_read_stat <- function(path, slice_position = 3, strip_after = NULL) {
  read_table2(path, col_names = FALSE) %>%
    arrange(X1) %>%
    mutate(
      sample_id = extract_sample(X1, slice_position, strip_after),
      gigabases = to_gigabases(X2)
    ) %>%
    select(sample_id, gigabases)
}

read_pair_stat <- function(path, slice_position = 3, suffix_to_remove = NULL) {
  read_table2(path, col_names = FALSE) %>%
    arrange(X1) %>%
    mutate(
      sample_id = extract_sample(X1, slice_position),
      sample_id = if (!is.null(suffix_to_remove)) str_remove(sample_id, fixed(suffix_to_remove)) else sample_id
    ) %>%
    select(sample_id, pairs = X2)
}

raw_reads <- read_read_stat("../stat/raw.fq.stat", strip_after = "_") %>%
  rename(Rawreads = gigabases)
dna_reads <- read_read_stat("../stat/dna.fq.stat") %>%
  rename(DNAreads = gigabases)
rna_reads <- read_read_stat("../stat/rna.fq.stat") %>%
  rename(RNAreads = gigabases)

raw_pairs <- read_pair_stat("../stat/raw.pairs.stat") %>%
  rename(raw_pairs = pairs)
pairs_dedup <- read_pair_stat("../stat/pairs.dedup.stat") %>%
  rename(pairs_dedup = pairs)

pairs_clean1 <- read_pair_stat("../stat/pairs.c1.stat", slice_position = 5, suffix_to_remove = ".pairs.gz") %>%
  rename(pairs_clean1 = pairs)
pairs_clean2 <- read_pair_stat("../stat/pairs.c12.stat", slice_position = 5, suffix_to_remove = ".pairs.gz") %>%
  rename(pairs_clean2 = pairs)
pairs_clean3 <- read_pair_stat("../stat/pairs.c123.stat", slice_position = 5, suffix_to_remove = ".pairs.gz") %>%
  rename(pairs_clean3 = pairs)
inter_pairs_clean3 <- read_pair_stat("../stat/inter.pairs.c123.stat", slice_position = 5, suffix_to_remove = ".pairs.gz") %>%
  rename(inter_pairs_clean3 = pairs)

yperx <- read_table2("../stat/yperx.stat", col_names = FALSE) %>%
  arrange(X1) %>%
  mutate(sample_id = extract_sample(X1, 2)) %>%
  select(sample_id, yperx = X2)

stat <- raw_reads %>%
  left_join(dna_reads, by = "sample_id") %>%
  left_join(rna_reads, by = "sample_id") %>%
  left_join(yperx, by = "sample_id") %>%
  left_join(raw_pairs, by = "sample_id") %>%
  left_join(pairs_dedup, by = "sample_id") %>%
  left_join(pairs_clean1, by = "sample_id") %>%
  left_join(pairs_clean2, by = "sample_id") %>%
  left_join(pairs_clean3, by = "sample_id") %>%
  left_join(inter_pairs_clean3, by = "sample_id")

rna_gene_counts <- read_table2("../result/RNA_Res/counts.gene.total.format.tsv")
rna_gene_matrix <- as.data.frame(rna_gene_counts %>% select(-gene))
feature_stat_gene <- tibble(
  sample_id = names(rna_gene_matrix),
  UMIs_gene = colSums(rna_gene_matrix),
  genes_gene = colSums(rna_gene_matrix != 0)
)

rna_exon_counts <- read_table2("../result/RNA_Res/counts.exon.total.format.tsv")
rna_exon_matrix <- as.data.frame(rna_exon_counts %>% select(-gene))
feature_stat_exon <- tibble(
  sample_id = names(rna_exon_matrix),
  UMIs_exon = colSums(rna_exon_matrix),
  genes_exon = colSums(rna_exon_matrix != 0)
)

if (config$if_RNA_snp_split) {
  genome1_counts <- read_table2("../result/RNA_Res/counts.gene.genome1.tsv")
  genome1_matrix <- as.data.frame(genome1_counts %>% select(-gene))
  genome2_counts <- read_table2("../result/RNA_Res/counts.gene.genome2.tsv")
  genome2_matrix <- as.data.frame(genome2_counts %>% select(-gene))

  feature_stat_genome1 <- tibble(
    sample_id = names(genome1_matrix),
    UMIs_gene_genome1 = colSums(genome1_matrix),
    genes_gene_genome1 = colSums(genome1_matrix != 0)
  )

  feature_stat_genome2 <- tibble(
    sample_id = names(genome2_matrix),
    UMIs_gene_genome2 = colSums(genome2_matrix),
    genes_gene_genome2 = colSums(genome2_matrix != 0)
  )

  stat <- stat %>%
    left_join(feature_stat_gene, by = "sample_id") %>%
    left_join(feature_stat_exon, by = "sample_id") %>%
    left_join(feature_stat_genome1, by = "sample_id") %>%
    left_join(feature_stat_genome2, by = "sample_id")
} else {
  stat <- stat %>%
    left_join(feature_stat_gene, by = "sample_id") %>%
    left_join(feature_stat_exon, by = "sample_id")
}

stat <- stat %>%
  rename(cellname = sample_id)

if (config$if_structure) {
  rmsd <- read_table2("../stat/rmsd.info", col_names = FALSE) %>%
    mutate(
      path_parts = str_split(X1, "/", simplify = TRUE),
      cellname = path_parts[, 1],
      res = str_split(path_parts[, 3], fixed("."), simplify = TRUE)[, 2],
      TOP3_RMSD = X5
    ) %>%
    select(cellname, res, TOP3_RMSD) %>%
    pivot_wider(names_from = res, values_from = TOP3_RMSD) %>%
    arrange(cellname)

  stat <- stat %>% left_join(rmsd, by = "cellname")
}

if (config$if_charm) {
  charm_reads <- map(
    names(config$split),
    function(split_name) {
      read_csv(paste0("../stat/", split_name, ".read.stat"), col_names = FALSE) %>%
        transmute(
          cellname = X1,
          !!paste0(split_name, "_reads") := X2 / 2 * 300 / 1e9
        )
    }
  ) %>%
    reduce(full_join, by = "cellname")

  stat <- stat %>% full_join(charm_reads, by = "cellname")
}


-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.4
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.4.4     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.0
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"`read_table2()` was deprecated in readr 2.0.0.
i Please use `read_table()` instead."

-- Column specification --------------------------------------------------------
cols(
  X1 = col_character(),
  X2 = col_double()
)


-- Column specification --------------------------------------------------------
cols(
  X1 = col_character(),
  X2 = col_double()
)


-- Column specification --------------------------------------------------------
cols(
  X1 = col_char

In [2]:
fill_numeric <- function(x) {
  x[is.na(x) | is.nan(x)] <- 0
  x
}

qc_metrics <- stat %>%
  mutate(
    RNAreadsRatio = RNAreads / (RNAreads + DNAreads),
    pairsPerRead = raw_pairs / DNAreads / 1e9 * 300,
    pairsValidRatio = pairs_clean3 / raw_pairs,
    interPairsRatio = inter_pairs_clean3 / pairs_clean3
  ) %>%
  mutate(across(where(is.numeric), fill_numeric))


In [3]:
qc_metrics


cellname,Rawreads,DNAreads,RNAreads,yperx,raw_pairs,pairs_dedup,pairs_clean1,pairs_clean2,pairs_clean3,...,1m,200k,20k,50k,ct_reads,atac_reads,RNAreadsRatio,pairsPerRead,pairsValidRatio,interPairsRatio
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
R1P10013,3.753487,3.752322,0.0010437,0.005022,3736619,779808,768346,508560,508059,...,4.232651,8.206307,20.8746,14.853,0.0063276,0.007428,0.0002780704,0.2987445,0.1359676,0.05423189


## Run below if this is a CHARM experiment

In [5]:
suppressPackageStartupMessages({
  library(Signac)
  library(Seurat)
  library(EnsDb.Mmusculus.v79)
  library(BSgenome.Mmusculus.UCSC.mm10)
  library(future)
})

plan("multicore", workers = 10)


In [ ]:
charm <- rna_gene_counts %>%
  column_to_rownames("gene") %>%
  as.matrix() %>%
  CreateSeuratObject(assay = "rna", min.cells = 0, min.features = 0)


In [ ]:
cell_names <- intersect(
  rownames(charm@meta.data),
  colnames(rna_gene_counts %>% select(-gene))
)


In [ ]:
atac_fragments <- CreateFragmentObject("../result/fragments/atac.fragments.bgz", cells = cell_names)
ct_fragments <- CreateFragmentObject("../result/fragments/ct.fragments.bgz", cells = cell_names)
mm10_genome <- seqlengths(BSgenome.Mmusculus.UCSC.mm10)
atac_count_matrix <- GenomeBinMatrix(atac_fragments, binsize = 5000, genome = mm10_genome)
ct_count_matrix <- GenomeBinMatrix(ct_fragments, binsize = 5000, genome = mm10_genome)

atac_assay <- CreateChromatinAssay(counts = atac_count_matrix, fragments = atac_fragments, genome = "mm10")
ct_assay <- CreateChromatinAssay(counts = ct_count_matrix, fragments = ct_fragments, genome = "mm10")

charm[["atac"]] <- atac_assay
charm[["ct"]] <- ct_assay


In [ ]:
charm@meta.data <- charm@meta.data %>%
  rownames_to_column("cellname")
rownames(charm@meta.data) <- charm@meta.data$cellname


In [ ]:
qc_metrics <- qc_metrics %>% full_join(
  charm@meta.data %>% dplyr::select(cellname, nCount_atac, nCount_ct, TSS.enrichment.atac, TSS.enrichment.ct),
  by = "cellname"
)


In [ ]:
qc_metrics %>% write_tsv("metadata_raw.tsv")
